### Deep Learning in the Eye Tracking World 
#### the tutorial presented during ETRA 2021 (https://etra.acm.org/2021/acceptedtutorials.html)
#### the code downloaded from: https://github.com/kasprowski/etra2021
@author: pawel@kasprowski.pl


# Model transforming a sequence of digits into a number
- input: sequence of three digits
- output: number in range(0,DATASE_SIZE)


Examples: 
- input: sequence [2,3,4] output: 234
- input: sequence [0,0,4] output: 4


In [2]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
import numpy as np

## Model

In [3]:
model = Sequential()
model.add(LSTM(128,input_shape=(None,1)))
model.add(Dense(1))
model.summary()

model.compile(loss='mean_absolute_error', optimizer="adam",metrics=['mae','mse'])
num_epochs = 0

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               66560     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________


## Dataset creation

In [4]:
DATASET_SIZE=256

samples = []
labels = []

for i in range(DATASET_SIZE):
    labels.append(i)
    samples.append( list(str(i).rjust(3,'0')) )

samples = np.array(samples,dtype=float)
samples = np.expand_dims(samples,axis=2)
labels = np.array(labels,dtype=int)

print("Sample (input):\n",samples[123])
print("Label (output):",labels[123])
print("Samples size",samples.shape)
print("Labels size",labels.shape)

Sample (input):
 [[1.]
 [2.]
 [3.]]
Label (output): 123
Samples size (256, 3, 1)
Labels size (256,)


In [5]:
TRAINING_SIZE = .5
from sklearn.model_selection import train_test_split
(trainSamples, testSamples, trainLabels, testLabels) = train_test_split(samples, labels,train_size=TRAINING_SIZE)
print('Training samples:',len(trainSamples),' test samples',len(testSamples))

Training samples: 128  test samples 128


In [7]:
EPOCHS=100
BATCH_SIZE = int(len(trainSamples)/4)
print('Training with',len(trainSamples),'samples',EPOCHS,'epochs and batch_size=',BATCH_SIZE)
for x in range(10):
    H = model.fit(trainSamples, trainLabels, epochs=EPOCHS,verbose=0,batch_size=BATCH_SIZE)
    num_epochs += EPOCHS
    print()
    print("Final epochs: {} - loss={:6.3f}, loss improvement={:6.3f}".
          format(num_epochs,H.history['loss'][-1], H.history['loss'][0]-H.history['loss'][-1]))
    check_model()
print("Done")

Training with 128 samples 100 epochs and batch_size= 32

Final epochs: 100 - loss=72.808, loss improvement=57.687
Average error: 71.131

Final epochs: 200 - loss=39.075, loss improvement=33.295
Average error: 38.942

Final epochs: 300 - loss=21.034, loss improvement=17.869
Average error: 21.549

Final epochs: 400 - loss=10.645, loss improvement=10.166
Average error: 11.140

Final epochs: 500 - loss= 4.984, loss improvement= 5.491
Average error:  5.483

Final epochs: 600 - loss= 3.193, loss improvement= 1.827
Average error:  3.097

Final epochs: 700 - loss= 1.649, loss improvement= 1.178
Average error:  1.666

Final epochs: 800 - loss= 0.921, loss improvement= 0.632
Average error:  1.018

Final epochs: 900 - loss= 0.674, loss improvement= 0.362
Average error:  0.641

Final epochs: 1000 - loss= 0.446, loss improvement= 0.156
Average error:  0.444
Done


In [8]:
def check_model(verbose=0,show_training=1):
    pred = model.predict(samples)
    for i in range(len(pred)):
        if(not show_training and i in trainLabels): continue
        p = pred[i][0]
        l = labels[i]
        if(verbose==1):
            train=''
            if i in trainLabels: train='[T]'
            print("{:2} > {:6.2f}, error ={:6.2f} {}".format(l,p,abs(p-l),train))
    avg_error = np.mean(np.absolute(pred-np.expand_dims(labels,axis=1)))
    print('Average error: {:6.3f}'.format(avg_error))
    return(avg_error)    
check_model(1)

 0 >   0.74, error =  0.74 
 1 >   0.95, error =  0.05 
 2 >   1.74, error =  0.26 [T]
 3 >   3.00, error =  0.00 
 4 >   4.29, error =  0.29 
 5 >   5.27, error =  0.27 
 6 >   6.04, error =  0.04 
 7 >   6.86, error =  0.14 [T]
 8 >   7.95, error =  0.05 [T]
 9 >   9.32, error =  0.32 
10 >   7.62, error =  2.38 
11 >   9.59, error =  1.41 
12 >  11.39, error =  0.61 
13 >  12.81, error =  0.19 [T]
14 >  13.92, error =  0.08 [T]
15 >  14.88, error =  0.12 [T]
16 >  15.83, error =  0.17 [T]
17 >  16.84, error =  0.16 [T]
18 >  17.88, error =  0.12 
19 >  18.93, error =  0.07 [T]
20 >  19.87, error =  0.13 [T]
21 >  20.83, error =  0.17 [T]
22 >  21.83, error =  0.17 
23 >  22.84, error =  0.16 
24 >  23.86, error =  0.14 [T]
25 >  24.88, error =  0.12 
26 >  25.86, error =  0.14 [T]
27 >  26.81, error =  0.19 
28 >  27.76, error =  0.24 
29 >  28.74, error =  0.26 
30 >  28.74, error =  1.26 
31 >  30.32, error =  0.68 
32 >  31.66, error =  0.34 [T]
33 >  32.78, error =  0.22 [T]
34 

0.44384736148640513

In [9]:
model.predict([[[2],[2],[1]]])

array([[220.40384]], dtype=float32)

In [ ]:
model.save('model_135_999.h5')